In [286]:
# Required Modules
import pandas as pd
import numpy as np


In [287]:
import warnings
warnings.filterwarnings("ignore")

from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

from lightgbm import LGBMClassifier

In [401]:
data = pd.read_csv('Resources/Final_Data.csv')
data.head()

,Zip_Code,Date,Year,Month,Sale_Price,Interest_Rate,Property_Tax,Rent_Price,Household_Income,Rent_Affordability,...,FTE_Employed,Unemployed,Expense_Index,Average_Commute,Crime_Index,Loan_Amount,Loan_Term,Loan_R,Loan_Payment,Home_Affordability
0,32003,2019-01-01,2019,1,244950.00,4.46,3778,1113,94154.00,0.14,...,10671,1069,104,35,28,195960.00,360.00,1.00,988.71,0.13
1,32003,2019-02-01,2019,2,270000.00,4.37,3778,1113,94154.00,0.14,...,10671,1069,104,35,28,216000.00,360.00,1.00,1077.82,0.14
2,32003,2019-03-01,2019,3,275000.00,4.26,3778,1113,94154.00,0.14,...,10671,1069,104,35,28,220000.00,360.00,1.00,1084.20,0.14
3,32003,2019-04-01,2019,4,264500.00,4.14,3778,1113,94154.00,0.14,...,10671,1069,104,35,28,211600.00,360.00,1.00,1027.67,0.13
4,32003,2019-05-01,2019,5,281000.00,4.07,3778,1113,94154.00,0.14,...,10671,1069,104,35,28,224800.00,360.00,1.00,1082.58,0.14


In [289]:
data.dtypes

Zip_Code                int64
Date                   object
Year                    int64
Month                   int64
Sale_Price            float64
Interest_Rate         float64
Property_Tax            int64
Rent_Price              int64
Household_Income      float64
Rent_Affordability    float64
Total_Sales             int64
FHA_Count               int64
Mobility_Rate         float64
School_Rating           int64
School_Grade           object
Owner_Occupied          int64
Renter_Occupied         int64
Total_Vacant            int64
Total_Dwellings         int64
FTE_Employed            int64
Unemployed              int64
Expense_Index           int64
Average_Commute         int64
Crime_Index             int64
Loan_Amount           float64
Loan_Term             float64
Loan_R                float64
Loan_Payment          float64
Home_Affordability    float64
dtype: object

In [402]:
ml_data = data[['Zip_Code', 'Year', 'Month', 'Mobility_Rate', 'Expense_Index', 'Crime_Index' , 'Total_Vacant', 'Total_Dwellings', 'Total_Sales', 'FHA_Count', 'Home_Affordability', 'Rent_Affordability', 'Sale_Price' ]]
ml_data

,Zip_Code,Year,Month,Mobility_Rate,Expense_Index,Crime_Index,Total_Vacant,Total_Dwellings,Total_Sales,FHA_Count,Home_Affordability,Rent_Affordability,Sale_Price
0,32003,2019,1,0.18,104,28,704,11724,32,7,0.13,0.14,244950.00
1,32003,2019,2,0.18,104,28,704,11724,41,2,0.14,0.14,270000.00
2,32003,2019,3,0.18,104,28,704,11724,21,7,0.14,0.14,275000.00
3,32003,2019,4,0.18,104,28,704,11724,42,3,0.13,0.14,264500.00
4,32003,2019,5,0.18,104,28,704,11724,66,8,0.14,0.14,281000.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
24466,34997,2021,1,0.14,91,113,2976,21551,81,10,0.20,0.25,305000.00
24467,34997,2021,2,0.14,91,113,2976,21551,116,7,0.20,0.25,300000.00
24468,34997,2021,2,0.14,91,113,2976,21551,116,7,0.20,0.25,300000.00
24469,34997,2021,3,0.14,91,113,2976,21551,7,12,0.42,0.25,599000.00


In [291]:
count = np.isinf(ml_data).values.sum()
print("It contains " + str(count) + " infinite values")

It contains 0 infinite values


In [292]:
ml_data_1mo = ml_data.sort_values(by = ['Zip_Code', 'Year', 'Month'], ascending = [True, True, True])
ml_data_1mo.head()

,Zip_Code,Year,Month,Mobility_Rate,Expense_Index,Crime_Index,Total_Vacant,Total_Dwellings,Total_Sales,FHA_Count,Home_Affordability,Rent_Affordability,Sale_Price
0,32003,2019,1,0.18,104,28,704,11724,32,7,0.13,0.14,244950.00
1,32003,2019,2,0.18,104,28,704,11724,41,2,0.14,0.14,270000.00
2,32003,2019,3,0.18,104,28,704,11724,21,7,0.14,0.14,275000.00
3,32003,2019,4,0.18,104,28,704,11724,42,3,0.13,0.14,264500.00
4,32003,2019,5,0.18,104,28,704,11724,66,8,0.14,0.14,281000.00


In [293]:
ml_data_1mo['Last_Month_Price'] = ml_data_1mo.groupby(['Zip_Code'])['Sale_Price'].shift()

price_difference = []

for column in ml_data_1mo[['Sale_Price', 'Last_Month_Price']]:
   # Select column contents by column name using [] operator
    current_sales = ml_data_1mo['Sale_Price'].values
    last_sales = ml_data_1mo['Last_Month_Price'].values
    price_difference.append(current_sales-last_sales)

ml_data_1mo['Last_Month_Diff'] = price_difference[0]

ml_data_1mo.head()

,Zip_Code,Year,Month,Mobility_Rate,Expense_Index,Crime_Index,Total_Vacant,Total_Dwellings,Total_Sales,FHA_Count,Home_Affordability,Rent_Affordability,Sale_Price,Last_Month_Price,Last_Month_Diff
0,32003,2019,1,0.18,104,28,704,11724,32,7,0.13,0.14,244950.00,nan,nan
1,32003,2019,2,0.18,104,28,704,11724,41,2,0.14,0.14,270000.00,244950.00,25050.00
2,32003,2019,3,0.18,104,28,704,11724,21,7,0.14,0.14,275000.00,270000.00,5000.00
3,32003,2019,4,0.18,104,28,704,11724,42,3,0.13,0.14,264500.00,275000.00,-10500.00
4,32003,2019,5,0.18,104,28,704,11724,66,8,0.14,0.14,281000.00,264500.00,16500.00


In [294]:
ml_data_2mo = ml_data.sort_values(by = ['Zip_Code', 'Year', 'Month'], ascending = [True, True, True])
ml_data_2mo.head()

,Zip_Code,Year,Month,Mobility_Rate,Expense_Index,Crime_Index,Total_Vacant,Total_Dwellings,Total_Sales,FHA_Count,Home_Affordability,Rent_Affordability,Sale_Price
0,32003,2019,1,0.18,104,28,704,11724,32,7,0.13,0.14,244950.00
1,32003,2019,2,0.18,104,28,704,11724,41,2,0.14,0.14,270000.00
2,32003,2019,3,0.18,104,28,704,11724,21,7,0.14,0.14,275000.00
3,32003,2019,4,0.18,104,28,704,11724,42,3,0.13,0.14,264500.00
4,32003,2019,5,0.18,104,28,704,11724,66,8,0.14,0.14,281000.00


In [295]:
ml_data_2mo['Last_Month_Price'] = ml_data_2mo.groupby(['Zip_Code'])['Sale_Price'].shift()
ml_data_2mo['Last_2Month_Price'] = ml_data_2mo.groupby(['Zip_Code'])['Sale_Price'].shift(2)


price_difference_1 = []
price_difference_2 = []

for column in ml_data_2mo[['Sale_Price', 'Last_Month_Price']]:
   # Select column contents by column name using [] operator
    current_sales = ml_data_2mo['Sale_Price'].values
    last_sales = ml_data_2mo['Last_Month_Price'].values
    last_2mo_sale = ml_data_2mo['Last_2Month_Price'].values
    price_difference.append(current_sales-last_sales)
    price_difference_2.append(current_sales-last_2mo_sale)

ml_data_2mo['Last_Month_Diff'] = price_difference[0]
ml_data_2mo['Last_2Month_Diff'] = price_difference_2[0]

ml_data_2mo.head()

,Zip_Code,Year,Month,Mobility_Rate,Expense_Index,Crime_Index,Total_Vacant,Total_Dwellings,Total_Sales,FHA_Count,Home_Affordability,Rent_Affordability,Sale_Price,Last_Month_Price,Last_2Month_Price,Last_Month_Diff,Last_2Month_Diff
0,32003,2019,1,0.18,104,28,704,11724,32,7,0.13,0.14,244950.00,nan,nan,nan,nan
1,32003,2019,2,0.18,104,28,704,11724,41,2,0.14,0.14,270000.00,244950.00,nan,25050.00,nan
2,32003,2019,3,0.18,104,28,704,11724,21,7,0.14,0.14,275000.00,270000.00,244950.00,5000.00,30050.00
3,32003,2019,4,0.18,104,28,704,11724,42,3,0.13,0.14,264500.00,275000.00,270000.00,-10500.00,-5500.00
4,32003,2019,5,0.18,104,28,704,11724,66,8,0.14,0.14,281000.00,264500.00,275000.00,16500.00,6000.00


In [414]:
ml_data_3mo = ml_data.sort_values(by = ['Zip_Code', 'Year', 'Month'], ascending = [True, True, True])
ml_data_3mo.head()

,Zip_Code,Year,Month,Mobility_Rate,Expense_Index,Crime_Index,Total_Vacant,Total_Dwellings,Total_Sales,FHA_Count,Home_Affordability,Rent_Affordability,Sale_Price
0,32003,2019,1,0.18,104,28,704,11724,32,7,0.13,0.14,244950.00
1,32003,2019,2,0.18,104,28,704,11724,41,2,0.14,0.14,270000.00
2,32003,2019,3,0.18,104,28,704,11724,21,7,0.14,0.14,275000.00
3,32003,2019,4,0.18,104,28,704,11724,42,3,0.13,0.14,264500.00
4,32003,2019,5,0.18,104,28,704,11724,66,8,0.14,0.14,281000.00


In [415]:
ml_data_3mo['Last_Month_Price'] = ml_data_3mo.groupby(['Zip_Code'])['Sale_Price'].shift()
ml_data_3mo['Future_Month_Price'] = ml_data_3mo.groupby(['Zip_Code'])['Sale_Price'].shift(-1)
ml_data_3mo['Future_2Month_Price'] = ml_data_3mo.groupby(['Zip_Code'])['Sale_Price'].shift(-2)
ml_data_3mo['Future_3Month_Price'] = ml_data_3mo.groupby(['Zip_Code'])['Sale_Price'].shift(-3)


# price_difference_1 = []
# price_difference_2 = []
# price_difference_3 = []


# for column in ml_data_3mo[['Sale_Price', 'Last_Month_Price', 'Future_Month_Price',
#                     'Future_2Month_Price', 'Future_3Month_Price']]:
#    # Select column contents by column name using [] operator
#     current_sales = ml_data_3mo['Sale_Price'].values
#     last_sales = ml_data_3mo['Future_Month_Price'].values
#     last_2mo_sale = ml_data_3mo['Future_2Month_Price'].values
#     last_3mo_sale = ml_data_3mo['Future_3Month_Price'].values
#     price_difference_1.append(current_sales-last_sales)
#     price_difference_2.append(current_sales-last_2mo_sale)
#     price_difference_3.append(current_sales-last_3mo_sale)
    

# ml_data_3mo['Last_Month_Diff'] = price_difference_1[0]
# ml_data_3mo['Last_2Month_Diff'] = price_difference_2[0]
# ml_data_3mo['Last_3Month_Diff'] = price_difference_3[0]

ml_data_3mo

,Zip_Code,Year,Month,Mobility_Rate,Expense_Index,Crime_Index,Total_Vacant,Total_Dwellings,Total_Sales,FHA_Count,Home_Affordability,Rent_Affordability,Sale_Price,Last_Month_Price,Future_Month_Price,Future_2Month_Price,Future_3Month_Price
0,32003,2019,1,0.18,104,28,704,11724,32,7,0.13,0.14,244950.00,nan,270000.00,275000.00,264500.00
1,32003,2019,2,0.18,104,28,704,11724,41,2,0.14,0.14,270000.00,244950.00,275000.00,264500.00,281000.00
2,32003,2019,3,0.18,104,28,704,11724,21,7,0.14,0.14,275000.00,270000.00,264500.00,281000.00,275000.00
3,32003,2019,4,0.18,104,28,704,11724,42,3,0.13,0.14,264500.00,275000.00,281000.00,275000.00,282500.00
4,32003,2019,5,0.18,104,28,704,11724,66,8,0.14,0.14,281000.00,264500.00,275000.00,282500.00,269577.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24466,34997,2021,1,0.14,91,113,2976,21551,81,10,0.20,0.25,305000.00,305000.00,300000.00,300000.00,599000.00
24467,34997,2021,2,0.14,91,113,2976,21551,116,7,0.20,0.25,300000.00,305000.00,300000.00,599000.00,599000.00
24468,34997,2021,2,0.14,91,113,2976,21551,116,7,0.20,0.25,300000.00,300000.00,599000.00,599000.00,nan
24469,34997,2021,3,0.14,91,113,2976,21551,7,12,0.42,0.25,599000.00,300000.00,599000.00,nan,nan


In [298]:
ml_data_4mo = ml_data.sort_values(by = ['Zip_Code', 'Year', 'Month'], ascending = [True, True, True])
ml_data_4mo.head()

,Zip_Code,Year,Month,Mobility_Rate,Expense_Index,Crime_Index,Total_Vacant,Total_Dwellings,Total_Sales,FHA_Count,Home_Affordability,Rent_Affordability,Sale_Price
0,32003,2019,1,0.18,104,28,704,11724,32,7,0.13,0.14,244950.00
1,32003,2019,2,0.18,104,28,704,11724,41,2,0.14,0.14,270000.00
2,32003,2019,3,0.18,104,28,704,11724,21,7,0.14,0.14,275000.00
3,32003,2019,4,0.18,104,28,704,11724,42,3,0.13,0.14,264500.00
4,32003,2019,5,0.18,104,28,704,11724,66,8,0.14,0.14,281000.00


In [299]:
ml_data_4mo['Last_Month_Price'] = ml_data_4mo.groupby(['Zip_Code'])['Sale_Price'].shift()
ml_data_4mo['Last_2Month_Price'] = ml_data_4mo.groupby(['Zip_Code'])['Sale_Price'].shift(2)
ml_data_4mo['Last_3Month_Price'] = ml_data_4mo.groupby(['Zip_Code'])['Sale_Price'].shift(3)
ml_data_4mo['Last_4Month_Price'] = ml_data_4mo.groupby(['Zip_Code'])['Sale_Price'].shift(4)


price_difference_1 = []
price_difference_2 = []
price_difference_3 = []
price_difference_4 = []


for column in ml_data_4mo[['Sale_Price', 'Last_Month_Price', 'Last_2Month_Price',
                        'Last_3Month_Price']]:
   # Select column contents by column name using [] operator
    current_sales = ml_data_4mo['Sale_Price'].values
    last_sales = ml_data_4mo['Last_Month_Price'].values
    last_2mo_sale = ml_data_4mo['Last_2Month_Price'].values
    last_3mo_sale = ml_data_4mo['Last_3Month_Price'].values
    last_4mo_sale = ml_data_4mo['Last_4Month_Price'].values
    price_difference_1.append(current_sales-last_sales)
    price_difference_2.append(current_sales-last_2mo_sale)
    price_difference_3.append(current_sales-last_3mo_sale)
    price_difference_4.append(current_sales-last_4mo_sale)
    

ml_data_4mo['Last_Month_Diff'] = price_difference_1[0]
ml_data_4mo['Last_2Month_Diff'] = price_difference_2[0]
ml_data_4mo['Last_3Month_Diff'] = price_difference_3[0]
ml_data_4mo['Last_4Month_Diff'] = price_difference_4[0]

ml_data_4mo

,Zip_Code,Year,Month,Mobility_Rate,Expense_Index,Crime_Index,Total_Vacant,Total_Dwellings,Total_Sales,FHA_Count,...,Rent_Affordability,Sale_Price,Last_Month_Price,Last_2Month_Price,Last_3Month_Price,Last_4Month_Price,Last_Month_Diff,Last_2Month_Diff,Last_3Month_Diff,Last_4Month_Diff
0,32003,2019,1,0.18,104,28,704,11724,32,7,...,0.14,244950.00,nan,nan,nan,nan,nan,nan,nan,nan
1,32003,2019,2,0.18,104,28,704,11724,41,2,...,0.14,270000.00,244950.00,nan,nan,nan,25050.00,nan,nan,nan
2,32003,2019,3,0.18,104,28,704,11724,21,7,...,0.14,275000.00,270000.00,244950.00,nan,nan,5000.00,30050.00,nan,nan
3,32003,2019,4,0.18,104,28,704,11724,42,3,...,0.14,264500.00,275000.00,270000.00,244950.00,nan,-10500.00,-5500.00,19550.00,nan
4,32003,2019,5,0.18,104,28,704,11724,66,8,...,0.14,281000.00,264500.00,275000.00,270000.00,244950.00,16500.00,6000.00,11000.00,36050.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24466,34997,2021,1,0.14,91,113,2976,21551,81,10,...,0.25,305000.00,305000.00,297000.00,297000.00,259750.00,0.00,8000.00,8000.00,45250.00
24467,34997,2021,2,0.14,91,113,2976,21551,116,7,...,0.25,300000.00,305000.00,305000.00,297000.00,297000.00,-5000.00,-5000.00,3000.00,3000.00
24468,34997,2021,2,0.14,91,113,2976,21551,116,7,...,0.25,300000.00,300000.00,305000.00,305000.00,297000.00,0.00,-5000.00,-5000.00,3000.00
24469,34997,2021,3,0.14,91,113,2976,21551,7,12,...,0.25,599000.00,300000.00,300000.00,305000.00,305000.00,299000.00,299000.00,294000.00,294000.00


1 Month Data Testing

In [300]:
y_value = ml_data_1mo['Sale_Price'].values

x_values = ml_data_1mo.drop(columns='Sale_Price')

feature_names = x_values.columns

x_values
y_value

array([244950., 270000., 275000., ..., 300000., 599000., 599000.])

In [301]:

X_train, X_test, y_train, y_test = train_test_split(x_values, y_value, random_state=42)

In [302]:
print("Training set - Features: ", X_train.shape, "Target: ", y_train.shape)
print("Test set - Features: ", X_test.shape, "Target: ",y_test.shape)

Training set - Features:  (18353, 14) Target:  (18353,)
Test set - Features:  (6118, 14) Target:  (6118,)


In [217]:
import lazypredict
from lazypredict.Supervised import LazyRegressor

In [29]:
reg = LazyRegressor(verbose=0, ignore_warnings=False, custom_metric=None)
reg_models, reg_predictions = reg.fit(X_train, X_test, y_train, y_test)

print(reg_models)
print(reg_predictions)

100%|██████████| 42/42 [01:40<00:00,  2.39s/it]                               Adjusted R-Squared  R-Squared       RMSE  \
Model                                                                     
LassoLarsIC                                  0.97       0.97   25423.17   
LassoLarsCV                                  0.97       0.97   25423.17   
TransformedTargetRegressor                   0.97       0.97   25423.17   
LarsCV                                       0.97       0.97   25423.17   
Lars                                         0.97       0.97   25423.17   
LinearRegression                             0.97       0.97   25423.17   
Lasso                                        0.97       0.97   25423.25   
BayesianRidge                                0.97       0.97   25423.88   
Ridge                                        0.97       0.97   25424.66   
LassoLars                                    0.97       0.97   25430.60   
LassoCV                                      0.97    

In [177]:
price_change_1mo = []

for row in ml_data_1mo['Last_Month_Diff']:
    if row > 0:
        price_change_1mo.append(1)
    else:
        price_change_1mo.append(0)
ml_data_1mo['Price_Change_1mo'] = price_change_1mo

ml_data_1mo

,Zip_Code,Year,Month,Mobility_Rate,Expense_Index,Crime_Index,Total_Vacant,Total_Dwellings,Total_Sales,FHA_Count,Home_Affordability,Rent_Affordability,Sale_Price,Last_Month_Price,Last_Month_Diff,Price_Change_1mo
0,32003,2019,1,0.18,104,28,704,11724,32,7,0.13,0.14,244950.00,nan,nan,0
1,32003,2019,2,0.18,104,28,704,11724,41,2,0.14,0.14,270000.00,244950.00,25050.00,1
2,32003,2019,3,0.18,104,28,704,11724,21,7,0.14,0.14,275000.00,270000.00,5000.00,1
3,32003,2019,4,0.18,104,28,704,11724,42,3,0.13,0.14,264500.00,275000.00,-10500.00,0
4,32003,2019,5,0.18,104,28,704,11724,66,8,0.14,0.14,281000.00,264500.00,16500.00,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21759,34997,2020,10,0.14,91,113,2976,21551,139,17,0.19,0.25,285000.00,285000.00,0.00,0
21760,34997,2020,11,0.14,91,113,2976,21551,132,11,0.17,0.25,259750.00,285000.00,-25250.00,0
21761,34997,2020,11,0.14,91,113,2976,21551,132,11,0.17,0.25,259750.00,259750.00,0.00,0
21762,34997,2020,12,0.14,91,113,2976,21551,142,16,0.20,0.25,297000.00,259750.00,37250.00,1


In [178]:
ml_data_1mo = ml_data_1mo.drop(columns=['Zip_Code', 'Year', 'Month','Last_Month_Price', 'Last_Month_Diff'])
ml_data_1mo

,Mobility_Rate,Expense_Index,Crime_Index,Total_Vacant,Total_Dwellings,Total_Sales,FHA_Count,Home_Affordability,Rent_Affordability,Sale_Price,Price_Change_1mo
0,0.18,104,28,704,11724,32,7,0.13,0.14,244950.00,0
1,0.18,104,28,704,11724,41,2,0.14,0.14,270000.00,1
2,0.18,104,28,704,11724,21,7,0.14,0.14,275000.00,1
3,0.18,104,28,704,11724,42,3,0.13,0.14,264500.00,0
4,0.18,104,28,704,11724,66,8,0.14,0.14,281000.00,1
...,...,...,...,...,...,...,...,...,...,...,...
21759,0.14,91,113,2976,21551,139,17,0.19,0.25,285000.00,0
21760,0.14,91,113,2976,21551,132,11,0.17,0.25,259750.00,0
21761,0.14,91,113,2976,21551,132,11,0.17,0.25,259750.00,0
21762,0.14,91,113,2976,21551,142,16,0.20,0.25,297000.00,1


In [179]:
ml_data_1mo.replace([np.inf, -np.inf], np.nan, inplace=True)
ml_data_1mo.dropna(inplace=True)

In [131]:
y_value = ml_data_1mo['Price_Change_1mo'].values

x_values = ml_data_1mo.drop(columns='Price_Change_1mo')

feature_names = x_values.columns

x_values


,Mobility_Rate,Expense_Index,Crime_Index,Total_Sales,FHA_Count,Home_Affordability,Rent_Affordability,Sale_Price
0,0.182327,104,28,32,7,0.126012,0.141853,244950.0
1,0.182327,104,28,41,2,0.137369,0.141853,270000.0
2,0.182327,104,28,21,7,0.138182,0.141853,275000.0
3,0.182327,104,28,42,3,0.130978,0.141853,264500.0
4,0.182327,104,28,66,8,0.137976,0.141853,281000.0
...,...,...,...,...,...,...,...,...
21759,0.143471,91,113,139,17,0.192070,0.247189,285000.0
21760,0.143471,91,113,132,11,0.173497,0.247189,259750.0
21761,0.143471,91,113,132,11,0.173497,0.247189,259750.0
21762,0.143471,91,113,142,16,0.196301,0.247189,297000.0


In [132]:
X_train, X_test, y_train, y_test = train_test_split(x_values, y_value, random_state=42)

In [42]:
from lazypredict.Supervised import LazyClassifier

clf = LazyClassifier(verbose=0,ignore_warnings=False)
models, predictions = clf.fit(X_train, X_test, y_train, y_test)
models

 21%|██        | 6/29 [00:04<00:16,  1.42it/s]CategoricalNB model failed to execute
Negative values in data passed to CategoricalNB (input X)
 90%|████████▉ | 26/29 [01:09<00:09,  3.19s/it]StackingClassifier model failed to execute
__init__() missing 1 required positional argument: 'estimators'
100%|██████████| 29/29 [01:10<00:00,  2.43s/it]


,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken
Model,,,,,
RandomForestClassifier,0.65,0.65,0.65,0.65,1.98
LGBMClassifier,0.65,0.65,0.65,0.65,0.18
ExtraTreesClassifier,0.65,0.65,0.65,0.65,1.19
XGBClassifier,0.62,0.62,0.62,0.62,0.45
BaggingClassifier,0.62,0.62,0.62,0.62,0.59
SVC,0.62,0.62,0.62,0.62,6.74
AdaBoostClassifier,0.60,0.60,0.60,0.60,0.47
SGDClassifier,0.60,0.60,0.60,0.60,0.07
KNeighborsClassifier,0.60,0.60,0.60,0.60,0.23


In [133]:
LGBMCmodel = LGBMClassifier(boosting_type='gbdt', objective='binary', num_leaves= 1000,
                                max_depth=25, min_data_in_leaf=100,
                                learning_rate=0.003, n_estimators=15000)
LGBMCmodel.fit(X_train, y_train)

LGBMClassifier(learning_rate=0.003, max_depth=25, min_data_in_leaf=100,
               n_estimators=15000, num_leaves=1000, objective='binary')

In [134]:
predict_test = LGBMCmodel.predict(X_test)

accuracy=accuracy_score(predict_test, y_test)
print('LightGBM Model accuracy score: {0:0.4f}'.format(accuracy_score(y_test, predict_test)))

LightGBM Model accuracy score: 0.6514


In [135]:
predict_train = LGBMCmodel.predict(X_train)

print('Training-set accuracy score: {0:0.4f}'. format(accuracy_score(y_train, predict_train)))

Training-set accuracy score: 0.9246


In [136]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, predict_test)
print('Confusion matrix\n\n', cm)
print('\nTrue Positives(TP) = ', cm[0,0])
print('\nTrue Negatives(TN) = ', cm[1,1])
print('\nFalse Positives(FP) = ', cm[0,1])
print('\nFalse Negatives(FN) = ', cm[1,0])

Confusion matrix

 [[1829 1002]
 [ 895 1715]]

True Positives(TP) =  1829

True Negatives(TN) =  1715

False Positives(FP) =  1002

False Negatives(FN) =  895


In [180]:
rf = RandomForestClassifier(n_estimators=100, max_depth=25, n_jobs=10000, verbose=2)
rf.fit(X_train,y_train)

predict_rf = rf.predict(X_test)

<IPython.core.display.Javascript object>

[Parallel(n_jobs=10000)]: Using backend ThreadingBackend with 10000 concurrent workers.
building tree 1 of 100building tree 2 of 100building tree 3 of 100building tree 4 of 100building tree 5 of 100building tree 6 of 100building tree 7 of 100building tree 8 of 100building tree 9 of 100building tree 10 of 100building tree 11 of 100building tree 12 of 100building tree 13 of 100building tree 14 of 100building tree 15 of 100building tree 16 of 100building tree 17 of 100building tree 18 of 100building tree 19 of 100building tree 20 of 100building tree 21 of 100building tree 22 of 100building tree 23 of 100building tree 24 of 100building tree 25 of 100building tree 26 of 100building tree 27 of 100building tree 28 of 100building tree 29 of 100building tree 30 of 100building tree 31 of 100building tree 32 of 100







building tree 33 of 100building tree 34 of 100building tree 35 of 100building tree 36 of 100building tree 37 of 100building tree 38 of 100building tree 39 of 100







building

In [181]:
print("Accuracy:", metrics.accuracy_score(y_test, predict_rf))

Accuracy: 0.6583348649145377


2 month testing

In [182]:
price_change_1mo = []
price_change_2mo = []

for row in ml_data_2mo['Last_Month_Diff']:
    if row > 0:
        price_change_1mo.append(1)
    else:
        price_change_1mo.append(0)

for row in ml_data_2mo['Last_2Month_Diff']:
    if row > 0:
        price_change_2mo.append(1)
    else:
        price_change_2mo.append(0)
ml_data_2mo['Price_Change_1mo'] = price_change_1mo
ml_data_2mo['Price_Change_2mo'] = price_change_2mo

ml_data_2mo.head()

,Zip_Code,Year,Month,Mobility_Rate,Expense_Index,Crime_Index,Total_Vacant,Total_Dwellings,Total_Sales,FHA_Count,Home_Affordability,Rent_Affordability,Sale_Price,Last_Month_Price,Last_2Month_Price,Last_Month_Diff,Last_2Month_Diff,Price_Change_1mo,Price_Change_2mo
0,32003,2019,1,0.18,104,28,704,11724,32,7,0.13,0.14,244950.00,nan,nan,nan,nan,0,0
1,32003,2019,2,0.18,104,28,704,11724,41,2,0.14,0.14,270000.00,244950.00,nan,25050.00,nan,1,0
2,32003,2019,3,0.18,104,28,704,11724,21,7,0.14,0.14,275000.00,270000.00,244950.00,5000.00,30050.00,1,1
3,32003,2019,4,0.18,104,28,704,11724,42,3,0.13,0.14,264500.00,275000.00,270000.00,-10500.00,-5500.00,0,0
4,32003,2019,5,0.18,104,28,704,11724,66,8,0.14,0.14,281000.00,264500.00,275000.00,16500.00,6000.00,1,1


In [183]:
ml_data_2mo = ml_data_2mo.drop(columns=['Zip_Code', 'Year', 'Month','Last_Month_Diff',
                            'Last_Month_Price', 'Last_2Month_Price', 'Last_2Month_Diff',
                            'Price_Change_1mo',])
ml_data_2mo.head()

,Mobility_Rate,Expense_Index,Crime_Index,Total_Vacant,Total_Dwellings,Total_Sales,FHA_Count,Home_Affordability,Rent_Affordability,Sale_Price,Price_Change_2mo
0,0.18,104,28,704,11724,32,7,0.13,0.14,244950.00,0
1,0.18,104,28,704,11724,41,2,0.14,0.14,270000.00,0
2,0.18,104,28,704,11724,21,7,0.14,0.14,275000.00,1
3,0.18,104,28,704,11724,42,3,0.13,0.14,264500.00,0
4,0.18,104,28,704,11724,66,8,0.14,0.14,281000.00,1


In [184]:
ml_data_2mo.replace([np.inf, -np.inf], np.nan, inplace=True)
ml_data_2mo.dropna(inplace=True)

In [206]:
y_value = ml_data_2mo['Price_Change_2mo'].values

x_values = ml_data_2mo.drop(columns='Price_Change_2mo')

feature_names = x_values.columns

x_values

,Mobility_Rate,Expense_Index,Crime_Index,Total_Vacant,Total_Dwellings,Total_Sales,FHA_Count,Home_Affordability,Rent_Affordability,Sale_Price
0,0.18,104,28,704,11724,32,7,0.13,0.14,244950.00
1,0.18,104,28,704,11724,41,2,0.14,0.14,270000.00
2,0.18,104,28,704,11724,21,7,0.14,0.14,275000.00
3,0.18,104,28,704,11724,42,3,0.13,0.14,264500.00
4,0.18,104,28,704,11724,66,8,0.14,0.14,281000.00
...,...,...,...,...,...,...,...,...,...,...
21759,0.14,91,113,2976,21551,139,17,0.19,0.25,285000.00
21760,0.14,91,113,2976,21551,132,11,0.17,0.25,259750.00
21761,0.14,91,113,2976,21551,132,11,0.17,0.25,259750.00
21762,0.14,91,113,2976,21551,142,16,0.20,0.25,297000.00


In [207]:
X_train, X_test, y_train, y_test = train_test_split(x_values, y_value, random_state=42)

In [208]:
LGBMCmodel = LGBMClassifier(boosting_type='gbdt', objective='binary', num_leaves= 1000,
                                max_depth=25, min_data_in_leaf=100,
                                learning_rate=0.003, n_estimators=15000)
LGBMCmodel.fit(X_train, y_train)

LGBMClassifier(learning_rate=0.003, max_depth=25, min_data_in_leaf=100,
               n_estimators=15000, num_leaves=1000, objective='binary')

In [209]:
predict_test = LGBMCmodel.predict(X_test)

accuracy=accuracy_score(predict_test, y_test)
print('LightGBM Model accuracy score: {0:0.4f}'.format(accuracy_score(y_test, predict_test)))

LightGBM Model accuracy score: 0.6694


In [210]:
predict_train = LGBMCmodel.predict(X_train)

print('Training-set accuracy score: {0:0.4f}'. format(accuracy_score(y_train, predict_train)))

Training-set accuracy score: 0.9409


In [211]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, predict_test)
print('Confusion matrix\n\n', cm)
print('\nTrue Positives(TP) = ', cm[0,0])
print('\nTrue Negatives(TN) = ', cm[1,1])
print('\nFalse Positives(FP) = ', cm[0,1])
print('\nFalse Negatives(FN) = ', cm[1,0])

Confusion matrix

 [[1738  933]
 [ 866 1904]]

True Positives(TP) =  1738

True Negatives(TN) =  1904

False Positives(FP) =  933

False Negatives(FN) =  866


In [187]:
rf = RandomForestClassifier(n_estimators=100, max_depth=25, n_jobs=10000, verbose=2)
rf.fit(X_train,y_train)

predict_rf = rf.predict(X_test)

<IPython.core.display.Javascript object>

[Parallel(n_jobs=10000)]: Using backend ThreadingBackend with 10000 concurrent workers.
building tree 1 of 100building tree 2 of 100building tree 3 of 100building tree 4 of 100building tree 5 of 100building tree 6 of 100building tree 7 of 100building tree 8 of 100
building tree 9 of 100building tree 10 of 100building tree 11 of 100building tree 12 of 100building tree 13 of 100building tree 14 of 100building tree 15 of 100building tree 16 of 100building tree 17 of 100building tree 18 of 100building tree 19 of 100building tree 20 of 100building tree 21 of 100building tree 22 of 100building tree 23 of 100building tree 24 of 100building tree 25 of 100building tree 26 of 100building tree 27 of 100building tree 28 of 100building tree 29 of 100building tree 30 of 100building tree 31 of 100building tree 32 of 100building tree 33 of 100building tree 34 of 100building tree 35 of 100building tree 36 of 100
building tree 37 of 100




building tree 38 of 100building tree 39 of 100building tree 40 

In [188]:
print("Accuracy:", metrics.accuracy_score(y_test, predict_rf))

Accuracy: 0.68296269068186


3 month testing

In [416]:
predict_21 = ml_data_3mo[ml_data_3mo['Year'] == 2021]

predict_21

,Zip_Code,Year,Month,Mobility_Rate,Expense_Index,Crime_Index,Total_Vacant,Total_Dwellings,Total_Sales,FHA_Count,Home_Affordability,Rent_Affordability,Sale_Price,Last_Month_Price,Future_Month_Price,Future_2Month_Price,Future_3Month_Price
24,32003,2021,1,0.18,104,28,704,11724,39,6,0.12,0.14,299900.00,294000.00,333800.00,339950.00,470000.00
25,32003,2021,2,0.18,104,28,704,11724,30,8,0.14,0.14,333800.00,299900.00,339950.00,470000.00,nan
26,32003,2021,3,0.18,104,28,704,11724,70,4,0.15,0.14,339950.00,333800.00,470000.00,nan,nan
27,32003,2021,4,0.18,104,28,704,11724,5,8,0.20,0.14,470000.00,339950.00,nan,nan,nan
51,32008,2021,1,0.07,82,118,964,3247,6,0,0.15,0.29,132000.00,169500.00,53200.00,140000.00,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24466,34997,2021,1,0.14,91,113,2976,21551,81,10,0.20,0.25,305000.00,305000.00,300000.00,300000.00,599000.00
24467,34997,2021,2,0.14,91,113,2976,21551,116,7,0.20,0.25,300000.00,305000.00,300000.00,599000.00,599000.00
24468,34997,2021,2,0.14,91,113,2976,21551,116,7,0.20,0.25,300000.00,300000.00,599000.00,599000.00,nan
24469,34997,2021,3,0.14,91,113,2976,21551,7,12,0.42,0.25,599000.00,300000.00,599000.00,nan,nan


In [417]:
ml_data_3mo = ml_data_3mo[ml_data_3mo['Year'] < 2021]
ml_data_3mo.head()

,Zip_Code,Year,Month,Mobility_Rate,Expense_Index,Crime_Index,Total_Vacant,Total_Dwellings,Total_Sales,FHA_Count,Home_Affordability,Rent_Affordability,Sale_Price,Last_Month_Price,Future_Month_Price,Future_2Month_Price,Future_3Month_Price
0,32003,2019,1,0.18,104,28,704,11724,32,7,0.13,0.14,244950.00,nan,270000.00,275000.00,264500.00
1,32003,2019,2,0.18,104,28,704,11724,41,2,0.14,0.14,270000.00,244950.00,275000.00,264500.00,281000.00
2,32003,2019,3,0.18,104,28,704,11724,21,7,0.14,0.14,275000.00,270000.00,264500.00,281000.00,275000.00
3,32003,2019,4,0.18,104,28,704,11724,42,3,0.13,0.14,264500.00,275000.00,281000.00,275000.00,282500.00
4,32003,2019,5,0.18,104,28,704,11724,66,8,0.14,0.14,281000.00,264500.00,275000.00,282500.00,269577.00


In [385]:
price_change_3mo = []

for row in ml_data_3mo['Future_3Month_Diff']:
    if row > 0:
        price_change_3mo.append(1)
    else:
        price_change_3mo.append(0)
ml_data_3mo['Price_Change_3mo'] = price_change_3mo

ml_data_3mo

KeyError: 'Future_3Month_Diff'

In [418]:
# ml_data_3mo = ml_data_3mo.drop(columns=['Zip_Code', 'Year', 'Month', 'Last_Month_Price',
#                         'Last_2Month_Price', 'Last_2Month_Price', 'Last_Month_Diff',
#                         'Last_2Month_Diff', 'Last_3Month_Price', 'Last_3Month_Diff'])
# ml_data_3mo

ml_data_3mo = ml_data_3mo.drop(columns=['Zip_Code', 'Year', 'Month', 'Future_Month_Price', 'Future_2Month_Price' ])
ml_data_3mo

,Mobility_Rate,Expense_Index,Crime_Index,Total_Vacant,Total_Dwellings,Total_Sales,FHA_Count,Home_Affordability,Rent_Affordability,Sale_Price,Last_Month_Price,Future_3Month_Price
0,0.18,104,28,704,11724,32,7,0.13,0.14,244950.00,nan,264500.00
1,0.18,104,28,704,11724,41,2,0.14,0.14,270000.00,244950.00,281000.00
2,0.18,104,28,704,11724,21,7,0.14,0.14,275000.00,270000.00,275000.00
3,0.18,104,28,704,11724,42,3,0.13,0.14,264500.00,275000.00,282500.00
4,0.18,104,28,704,11724,66,8,0.14,0.14,281000.00,264500.00,269577.00
...,...,...,...,...,...,...,...,...,...,...,...,...
24460,0.14,91,113,2976,21551,139,17,0.19,0.25,285000.00,285000.00,297000.00
24461,0.14,91,113,2976,21551,132,11,0.17,0.25,259750.00,285000.00,297000.00
24462,0.14,91,113,2976,21551,132,11,0.17,0.25,259750.00,259750.00,305000.00
24463,0.14,91,113,2976,21551,142,16,0.20,0.25,297000.00,259750.00,305000.00


In [419]:
ml_data_3mo.replace([np.inf, -np.inf], np.nan, inplace=True)
ml_data_3mo.dropna(inplace=True)

In [420]:
y_value = ml_data_3mo['Future_3Month_Price'].values

x_values = ml_data_3mo.drop(columns='Future_3Month_Price')

feature_names = x_values.columns

x_values

,Mobility_Rate,Expense_Index,Crime_Index,Total_Vacant,Total_Dwellings,Total_Sales,FHA_Count,Home_Affordability,Rent_Affordability,Sale_Price,Last_Month_Price
1,0.18,104,28,704,11724,41,2,0.14,0.14,270000.00,244950.00
2,0.18,104,28,704,11724,21,7,0.14,0.14,275000.00,270000.00
3,0.18,104,28,704,11724,42,3,0.13,0.14,264500.00,275000.00
4,0.18,104,28,704,11724,66,8,0.14,0.14,281000.00,264500.00
5,0.18,104,28,704,11724,43,3,0.13,0.14,275000.00,281000.00
...,...,...,...,...,...,...,...,...,...,...,...
24460,0.14,91,113,2976,21551,139,17,0.19,0.25,285000.00,285000.00
24461,0.14,91,113,2976,21551,132,11,0.17,0.25,259750.00,285000.00
24462,0.14,91,113,2976,21551,132,11,0.17,0.25,259750.00,259750.00
24463,0.14,91,113,2976,21551,142,16,0.20,0.25,297000.00,259750.00


In [410]:
X_train, X_test, y_train, y_test = train_test_split(x_values, y_value, random_state=42)

In [232]:
reg = LazyRegressor(verbose=0, ignore_warnings=False, custom_metric=None)
reg_models, reg_predictions = reg.fit(X_train, X_test, y_train, y_test)

print(reg_models)
print(reg_predictions)

100%|██████████| 42/42 [01:48<00:00,  2.59s/it]                               Adjusted R-Squared  R-Squared       RMSE  \
Model                                                                     
XGBRegressor                                 0.92       0.92   43650.01   
RandomForestRegressor                        0.91       0.91   46051.42   
BaggingRegressor                             0.91       0.91   47400.16   
ExtraTreesRegressor                          0.91       0.91   47992.78   
GradientBoostingRegressor                    0.90       0.90   49019.74   
ExtraTreeRegressor                           0.89       0.89   52563.07   
LGBMRegressor                                0.88       0.88   55091.57   
KNeighborsRegressor                          0.87       0.87   56417.21   
HistGradientBoostingRegressor                0.87       0.87   56524.50   
DecisionTreeRegressor                        0.86       0.86   59701.64   
LinearRegression                             0.83    

In [347]:
print(len(X_test))
print(len(y_test))

6118
6118


In [340]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression()
classifier

LogisticRegression()

In [341]:
classifier.fit(X_train, y_train)

LogisticRegression()

In [342]:
print(f"Training Data Score: {classifier.score(X_train, y_train)}")
print(f"Testing Data Score: {classifier.score(X_test, y_test)}")

Training Data Score: 0.5946166839208848
Testing Data Score: 0.5823798627002288


In [421]:
from sklearn.ensemble import RandomForestRegressor

rf_regr = RandomForestRegressor(n_estimators=1000, n_jobs=-1, random_state=42)
rf_regr.fit(X_train, y_train)

RandomForestRegressor(n_estimators=1000, n_jobs=-1, random_state=42)

In [422]:
print(f"Training Data Score: {rf_regr.score(X_train, y_train)}")
print(f"Testing Data Score: {rf_regr.score(X_test, y_test)}")

Training Data Score: 0.9690603920160642
Testing Data Score: 0.7905362208246793


In [423]:
from sklearn.metrics import mean_squared_log_error
def rmsle(ytrue, ypred):
    return np.sqrt(mean_squared_log_error(ytrue, ypred))

In [424]:
rf_pred = rf_regr.predict(X_test)

error = rmsle(y_test, rf_pred)
    
print('Mean Error = %.5f' % np.mean(error))

Mean Error = 0.27486


In [425]:
from lightgbm import LGBMRegressor

In [426]:
lgbm_regr = LGBMRegressor(n_estimators=1000, learning_rate=0.001)
lgbm_regr.fit(X_train, y_train)

LGBMRegressor(learning_rate=0.001, n_estimators=1000)

In [427]:
print(f"Training Data Score: {lgbm_regr.score(X_train, y_train)}")
print(f"Testing Data Score: {lgbm_regr.score(X_test, y_test)}")

Training Data Score: 0.6898486182466572
Testing Data Score: 0.664151329481192


In [314]:
predict_test = LGBMCmodel.predict(X_test)

accuracy=accuracy_score(predict_test, y_test)
print('LightGBM Model accuracy score: {0:0.4f}'.format(accuracy_score(y_test, predict_test)))

ValueError: Number of features of the model must match the input. Model n_features_ is 10 and input n_features is 15 

In [192]:
y_value = ml_data_3mo['Price_Change_3mo'].values

x_values = ml_data_3mo.drop(columns='Price_Change_3mo')

feature_names = x_values.columns

x_values
y_value

array([0, 0, 0, ..., 0, 1, 1])

In [193]:
X_train, X_test, y_train, y_test = train_test_split(x_values, y_value, random_state=42)

In [169]:
from lazypredict.Supervised import LazyClassifier

clf = LazyClassifier(verbose=0,ignore_warnings=False)
models, predictions = clf.fit(X_train, X_test, y_train, y_test)
models

 21%|██        | 6/29 [00:05<00:17,  1.30it/s]CategoricalNB model failed to execute
Negative values in data passed to CategoricalNB (input X)
 93%|█████████▎| 27/29 [01:23<00:06,  3.17s/it]StackingClassifier model failed to execute
__init__() missing 1 required positional argument: 'estimators'
100%|██████████| 29/29 [01:24<00:00,  2.91s/it]


,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken
Model,,,,,
RandomForestClassifier,0.66,0.66,0.66,0.66,2.17
ExtraTreesClassifier,0.65,0.65,0.65,0.65,1.38
XGBClassifier,0.64,0.64,0.64,0.64,0.65
LGBMClassifier,0.64,0.64,0.64,0.64,0.55
BaggingClassifier,0.62,0.62,0.62,0.62,0.66
SVC,0.62,0.62,0.62,0.62,7.64
LabelSpreading,0.61,0.61,0.61,0.61,11.26
LabelPropagation,0.61,0.61,0.61,0.61,7.39
NuSVC,0.60,0.60,0.60,0.60,46.45


In [156]:
from lightgbm import LGBMClassifier

In [157]:
y_value = ml_data_3mo['Price_Change_3mo']

x_values = ml_data_3mo.drop(columns='Price_Change_3mo')

feature_names = x_values.columns

x_values
y_value

0        0
1        0
2        0
3        1
4        1
        ..
21759    1
21760    1
21761    0
21762    1
21763    1
Name: Price_Change_3mo, Length: 21764, dtype: int64

In [158]:
X_train, X_test, y_train, y_test = train_test_split(x_values, y_value, random_state=42)

In [159]:
LGBMCmodel = LGBMClassifier(boosting_type='gbdt', objective='binary', num_leaves= 1000,
                                max_depth=25, min_data_in_leaf=100,
                                learning_rate=0.003, n_estimators=15000)
LGBMCmodel.fit(X_train, y_train)

LGBMClassifier(learning_rate=0.003, max_depth=25, min_data_in_leaf=100,
               n_estimators=15000, num_leaves=1000, objective='binary')

In [160]:
predict_test = LGBMCmodel.predict(X_test)

accuracy=accuracy_score(predict_test, y_test)
print('LightGBM Model accuracy score: {0:0.4f}'.format(accuracy_score(y_test, predict_test)))

LightGBM Model accuracy score: 0.6558


In [161]:
predict_train = LGBMCmodel.predict(X_train)

print('Training-set accuracy score: {0:0.4f}'. format(accuracy_score(y_train, predict_train)))

Training-set accuracy score: 0.9339


In [252]:
print('Training set score: {:.4f}'.format(LGBMCmodel.score(X_train, y_train)))

print('Test set score: {:.4f}'.format(LGBMCmodel.score(X_test, y_test)))

Training set score: 0.9246
Test set score: 0.6514


In [162]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, predict_test)
print('Confusion matrix\n\n', cm)
print('\nTrue Positives(TP) = ', cm[0,0])
print('\nTrue Negatives(TN) = ', cm[1,1])
print('\nFalse Positives(FP) = ', cm[0,1])
print('\nFalse Negatives(FN) = ', cm[1,0])

Confusion matrix

 [[1748  930]
 [ 943 1820]]

True Positives(TP) =  1748

True Negatives(TN) =  1820

False Positives(FP) =  930

False Negatives(FN) =  943


In [194]:
rf = RandomForestClassifier(n_estimators=100, max_depth=25, n_jobs=10000, verbose=2)
rf.fit(X_train,y_train)

predict_rf = rf.predict(X_test)

<IPython.core.display.Javascript object>

[Parallel(n_jobs=10000)]: Using backend ThreadingBackend with 10000 concurrent workers.
building tree 1 of 100building tree 2 of 100building tree 3 of 100building tree 4 of 100building tree 5 of 100building tree 6 of 100building tree 7 of 100building tree 8 of 100building tree 9 of 100building tree 10 of 100building tree 11 of 100building tree 12 of 100building tree 13 of 100building tree 14 of 100building tree 15 of 100building tree 16 of 100building tree 17 of 100building tree 18 of 100building tree 19 of 100building tree 20 of 100building tree 21 of 100building tree 22 of 100building tree 23 of 100building tree 24 of 100building tree 25 of 100building tree 26 of 100building tree 27 of 100building tree 28 of 100building tree 29 of 100building tree 30 of 100building tree 31 of 100building tree 32 of 100building tree 33 of 100building tree 34 of 100building tree 35 of 100building tree 36 of 100building tree 37 of 100building tree 38 of 100

building tree 39 of 100
building tree 40 of 1

In [195]:
print("Accuracy:", metrics.accuracy_score(y_test, predict_rf))

Accuracy: 0.668627090608344


In [199]:
price_change_4mo = []

for row in ml_data_4mo['Last_4Month_Diff']:
    if row > 0:
        price_change_4mo.append(1)
    else:
        price_change_4mo.append(0)
ml_data_4mo['Price_Change_4mo'] = price_change_4mo

ml_data_4mo

,Zip_Code,Year,Month,Mobility_Rate,Expense_Index,Crime_Index,Total_Vacant,Total_Dwellings,Total_Sales,FHA_Count,...,Sale_Price,Last_Month_Price,Last_2Month_Price,Last_3Month_Price,Last_4Month_Price,Last_Month_Diff,Last_2Month_Diff,Last_3Month_Diff,Last_4Month_Diff,Price_Change_4mo
0,32003,2019,1,0.18,104,28,704,11724,32,7,...,244950.00,nan,nan,nan,nan,nan,nan,nan,nan,0
1,32003,2019,2,0.18,104,28,704,11724,41,2,...,270000.00,244950.00,nan,nan,nan,25050.00,nan,nan,nan,0
2,32003,2019,3,0.18,104,28,704,11724,21,7,...,275000.00,270000.00,244950.00,nan,nan,5000.00,30050.00,nan,nan,0
3,32003,2019,4,0.18,104,28,704,11724,42,3,...,264500.00,275000.00,270000.00,244950.00,nan,-10500.00,-5500.00,19550.00,nan,0
4,32003,2019,5,0.18,104,28,704,11724,66,8,...,281000.00,264500.00,275000.00,270000.00,244950.00,16500.00,6000.00,11000.00,36050.00,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21759,34997,2020,10,0.14,91,113,2976,21551,139,17,...,285000.00,285000.00,256000.00,256000.00,251000.00,0.00,29000.00,29000.00,34000.00,1
21760,34997,2020,11,0.14,91,113,2976,21551,132,11,...,259750.00,285000.00,285000.00,256000.00,256000.00,-25250.00,-25250.00,3750.00,3750.00,1
21761,34997,2020,11,0.14,91,113,2976,21551,132,11,...,259750.00,259750.00,285000.00,285000.00,256000.00,0.00,-25250.00,-25250.00,3750.00,1
21762,34997,2020,12,0.14,91,113,2976,21551,142,16,...,297000.00,259750.00,259750.00,285000.00,285000.00,37250.00,37250.00,12000.00,12000.00,1


In [200]:
ml_data_4mo = ml_data_4mo.drop(columns=['Zip_Code', 'Year', 'Month', 'Last_Month_Price',
                        'Last_2Month_Price', 'Last_2Month_Price', 'Last_Month_Diff',
                        'Last_2Month_Diff', 'Last_3Month_Price', 'Last_3Month_Diff',
                        'Last_4Month_Diff', 'Last_4Month_Price'])
ml_data_4mo

,Mobility_Rate,Expense_Index,Crime_Index,Total_Vacant,Total_Dwellings,Total_Sales,FHA_Count,Home_Affordability,Rent_Affordability,Sale_Price,Price_Change_4mo
0,0.18,104,28,704,11724,32,7,0.13,0.14,244950.00,0
1,0.18,104,28,704,11724,41,2,0.14,0.14,270000.00,0
2,0.18,104,28,704,11724,21,7,0.14,0.14,275000.00,0
3,0.18,104,28,704,11724,42,3,0.13,0.14,264500.00,0
4,0.18,104,28,704,11724,66,8,0.14,0.14,281000.00,1
...,...,...,...,...,...,...,...,...,...,...,...
21759,0.14,91,113,2976,21551,139,17,0.19,0.25,285000.00,1
21760,0.14,91,113,2976,21551,132,11,0.17,0.25,259750.00,1
21761,0.14,91,113,2976,21551,132,11,0.17,0.25,259750.00,1
21762,0.14,91,113,2976,21551,142,16,0.20,0.25,297000.00,1


In [201]:
ml_data_3mo.replace([np.inf, -np.inf], np.nan, inplace=True)
ml_data_3mo.dropna(inplace=True)

In [202]:
y_value = ml_data_3mo['Price_Change_3mo'].values

x_values = ml_data_3mo.drop(columns='Price_Change_3mo')

feature_names = x_values.columns

x_values
y_value

array([0, 0, 0, ..., 0, 1, 1])

In [203]:
X_train, X_test, y_train, y_test = train_test_split(x_values, y_value, random_state=42)

In [204]:
rf = RandomForestClassifier(n_estimators=100, max_depth=25, n_jobs=10000, verbose=2)
rf.fit(X_train,y_train)

predict_rf = rf.predict(X_test)

<IPython.core.display.Javascript object>

[Parallel(n_jobs=10000)]: Using backend ThreadingBackend with 10000 concurrent workers.
building tree 1 of 100building tree 2 of 100building tree 3 of 100building tree 4 of 100building tree 5 of 100building tree 6 of 100building tree 7 of 100building tree 8 of 100building tree 9 of 100building tree 10 of 100building tree 11 of 100building tree 12 of 100building tree 13 of 100building tree 14 of 100building tree 15 of 100building tree 16 of 100building tree 17 of 100building tree 18 of 100building tree 19 of 100building tree 20 of 100building tree 21 of 100building tree 22 of 100building tree 23 of 100building tree 24 of 100building tree 25 of 100building tree 26 of 100building tree 27 of 100building tree 28 of 100building tree 29 of 100building tree 30 of 100building tree 31 of 100building tree 32 of 100building tree 33 of 100building tree 34 of 100building tree 35 of 100building tree 36 of 100building tree 37 of 100building tree 38 of 100

building tree 39 of 100building tree 40 of 10

In [205]:
print("Accuracy:", metrics.accuracy_score(y_test, predict_rf))

Accuracy: 0.6706487777981989
